In [1]:
%reload_ext autoreload
%autoreload 2

import torch
print(torch.__version__)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.is_available()

import json
from pathlib import Path
from typing import Iterator
import time

import datasets
import matplotlib.pyplot as plt
from tokenizers import BertWordPieceTokenizer, Regex, normalizers
from tqdm import tqdm
from transformers import (
    BertConfig,
    BertForMaskedLM,
    BertTokenizerFast,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
import math
from functools import reduce
from collections import namedtuple
from torch import nn
import torch.nn.functional as F
# from reformer_pytorch import ReformerLM
# from x_transformers import TransformerWrapper, Decoder,Encoder
from torch.utils.data import DataLoader
import math 
from accelerate import Accelerator
import torch.nn.functional as F
# from einops import rearrange, pack, unpack
import sys
import os
#get cur dir
cur_dir = os.getcwd()
if 'miu' in cur_dir:
    sys.path.append('/home/miu/Projects/NLPLab/')
elif 'kiki' in cur_dir:
    sys.path.append('/home/kiki/dados_servidor/Servidor/NLPLab/')
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"


2.0.1+cu117


/home/miu/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_training='hf'#'bert' , 'tokemonster
# LIMIT_DATASET = 2016 * 4  # keep small for development, set to None for full dataset
#(3090) Para 128 tokens(Bert) e 32000 vocab: 1000 = 6 seg, 10000 = 1min, 100000 = 10min, 1m = 100min, 10m = 16h, 100m = 160h ou 6.6 dias
#(3090) Para 84 tokens(tokenmonster salva 35%) e 24000 vocab(Bert): 1000 = 4.3s, 10000 = 43s, 100000 = 7.1m, 1m = 71min, 10m = 11.8h, 100m = 118h ou 5 dias
#(4090) Para 84 tokens(tokenmonster salva 35%) e 24000 vocab(Electra): 1000 = 3.5s, 10000 = 35s, 100000 = 5.9m, 1m = 59min, 10m = 9.8h, 100m = 98h ou 4 dias
#(4090) Para 84 tokens(tokenmonster salva 35%) e 16000 vocab (Electra): 1000 = 2.1s, 10000 = 21s, 100000 = 3.53m, 1m = 35.3min, 10m = 5.8h, 100m = 58h ou 2.4 dias

#com retnet. Confirmado que é O(n). Se lembrando que não existe mta vantagem além do custo crescer sequencialmente, já que o conteudo 
#(4090) Para 84 tokens(tokenmonster salva 35%) batch size 200 e 16000 vocab (Electra): 1000 = 2.1s, 10000 = 21s, 100000 = 3.53m, 1m = 35.3min, 10m = 5.8h, 100m = 58h ou 2.4 dias
#(4090) Para 168 tokens(tokenmonster salva 35%) batch size 100 e 16000 vocab (Electra): 1000 = 3.7s, 10000 = 37.5s,
#(4090) Para 1000 tokens(tokenmonster salva 35%) batch size 12 e 16000 vocab (Electra):            , 10000 = 4min32s,


#HIPOTÉTICO(Se conseguisse representar um texto com menos tokens):
#(4090) Para 24 tokens(tokenmonster salva 35%)(Electra): 1000 = , 10000 = , 100000 = 3.19m, 1m = 32min, 10m = 6.5h, 100m = 65h ou 2.7 dias
#HIPOTÉTICO(Se diminuisse o vocabulario para 1024 tokens diferentes)
#(4090) Para 84 tokens(tokenmonster salva 35%) e 1024 vocab: 1000 =1.6s , 10000 = 16.2s,

#losses:
#1m dataset, electra, tokenmonster: iteration:  7800 , total_loss:  26.277032788594564
#10m dataset, 84 tokens(tokenmonster salva 35%) e 16000 vocab (Electra) iteration:  49980 , total_loss:  20.8733185450236. 339min total


LIMIT_DATASET = 1_000_000
RANDOM_SEED = 42
NUM_TOKENIZER_TRAINING_ITEMS = 1_000_000  # I made this up, but it seems reasonable
# VOCAB_SIZE = 32_768  # from Cramming
# DEVICE_BATCH_SIZE = 100 # aprox 128, adjust to get near 100% gpu memory use
# MODEL_MAX_SEQ_LEN = 128  # from Cramming

VOCAB_SIZE = 32_768  # tokenmonster
# VOCAB_SIZE = 1_024  # tokenmonster
DEVICE_BATCH_SIZE = 12 # Token monster aguenta um batch size de (200-248)!! Geralmente melhora a qualidade do treino
MODEL_MAX_SEQ_LEN = 512 # token_monster


MASK_ID=4
PAD_ID=0
gradient_accumulation_steps = 2048 // DEVICE_BATCH_SIZE  # roughly based on Cramming
batch_size = DEVICE_BATCH_SIZE * gradient_accumulation_steps
print('batch_size: ',batch_size)
RUN_DIR = Path("data") / f"run_{time.strftime('%Y%m%d-%H%M%S')}"
CHECKPOINT_DIR = RUN_DIR / "training_checkpoints"
MODEL_DIR = RUN_DIR / "model"
TOKENIZER_PATH = RUN_DIR / "tokenizer.json"
TRAINER_HISTORY_PATH = RUN_DIR / "trainer_history.json"

RUN_DIR.mkdir(exist_ok=True, parents=True)

dataset = datasets.load_dataset(
    "sradc/chunked-shuffled-wikipedia20220301en-bookcorpusopen",
    split=f"train[:{LIMIT_DATASET}]" if LIMIT_DATASET else "train",
    revision="0e6fada2dd43136e4a3f637da41de2e596aee674",
)
print('loaded dataset!!')

from utils.process_tokenizer import get_tokenizer
# tokenizer,tokenized_dataset,norm,vocab=get_tokenizer(dataset,NUM_TOKENIZER_TRAINING_ITEMS,VOCAB_SIZE,TOKENIZER_PATH,'clm',MODEL_MAX_SEQ_LEN+2)
tokenized_dataset,norm,vocab,tokenizer=get_tokenizer(dataset,NUM_TOKENIZER_TRAINING_ITEMS,VOCAB_SIZE,TOKENIZER_PATH,'mlm',MODEL_MAX_SEQ_LEN+2)
# tokenized_dataset,norm,vocab=get_tokenizer(dataset,NUM_TOKENIZER_TRAINING_ITEMS,VOCAB_SIZE,TOKENIZER_PATH,'smlm',MODEL_MAX_SEQ_LEN+2)
MASK_ID=vocab.tokenize("[MASK]")[0]
N_DOMAINS=3


batch_size:  2040


Resolving data files: 100%|██████████| 53/53 [00:01<00:00, 48.50it/s]


loaded dataset!!


In [3]:
from x_transformers import TransformerWrapper, Decoder,Encoder


#encoder decoder
colt5_dict={
            'encoder':{'is_colt5_encoder':True,'light_ff_mult':0.5,'heavy_ff_mult':4,'num_heavy_tokens':30,
                   'light_dim_head':64,'light_heads':8,'light_window_size':128,
                   'heavy_dim_head':64,'heavy_heads':8,'num_heavy_tokens_q':30,
                   'num_heavy_tokens_kv':30},
            }

# model = TransformerWrapper(
model = TransformerWrapper(
    num_tokens = VOCAB_SIZE+8,
    max_seq_len = MODEL_MAX_SEQ_LEN,
    attn_layers = Encoder(
        dim = int(768),
        depth = 12,
        heads =12,
        attn_flash = True,
        colt5_dict=colt5_dict['encoder'],
        # rel_pos_bias = True
    ),
    emb_dim=128,
)

In [4]:
# input_ids are the indices corresponding to each token in the sentence.
# attention_mask indicates whether a token should be attended to or not.
# token_type_ids identifies which sequence a token belongs to when there is more than one sequence

train_dataloader = DataLoader(
        tokenized_dataset, shuffle=True, batch_size=DEVICE_BATCH_SIZE
    )
from utils.utils import get_optimizer_scheduler
optimizer,lr_scheduler,max_train_steps = get_optimizer_scheduler(model,train_dataloader,gradient_accumulation_steps,learning_rate=5e-5,weight_decay=0, num_warmup_steps=0, max_train_steps=None,lr_scheduler_type='linear',num_train_epochs=1)

accelerator = Accelerator()
# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, lr_scheduler
)

In [5]:
# Only show the progress bar once on each machine.
# progress_bar = tqdm(range(max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
starting_epoch = 0

# update the progress_bar if load from checkpoint
total_loss=0
# progress_bar.update(completed_steps)
steps_log=30
count_amostra=0
num_train_epochs=1
for epoch in range(starting_epoch, num_train_epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):

        # with accelerator.accumulate(model):
       
        logits=model(batch['input_ids']) 
        loss = F.cross_entropy(logits.transpose(1, 2),batch['input_ids'],ignore_index = PAD_ID)
        count_amostra+=int(len(batch['input_ids']))
        # print(loss)
        total_loss += loss.detach().float().cpu().numpy().item()
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()       
        
        if step%steps_log==0:
            print('iteration: ',step,', total_loss: ',total_loss/steps_log)
            print('count_amostra:',count_amostra)
            total_loss=0
    # Checks if the accelerator has performed an optimization step behind the scenes
    if accelerator.sync_gradients:
        # progress_bar.update(1)
        completed_steps += 1

    if completed_steps >= max_train_steps:
        break

    model.eval()
    losses = []


iteration:  0 , total_loss:  0.3521053949991862
count_amostra: 12
iteration:  30 , total_loss:  8.650077470143636
count_amostra: 372
iteration:  60 , total_loss:  7.599205573399861
count_amostra: 732
iteration:  90 , total_loss:  7.072017113367717
count_amostra: 1092
iteration:  120 , total_loss:  6.944790649414062
count_amostra: 1452
iteration:  150 , total_loss:  6.80439354578654
count_amostra: 1812
iteration:  180 , total_loss:  6.712727657953898
count_amostra: 2172
iteration:  210 , total_loss:  6.577449289957682
count_amostra: 2532
iteration:  240 , total_loss:  6.426461887359619
count_amostra: 2892
iteration:  270 , total_loss:  6.316332403818766
count_amostra: 3252
iteration:  300 , total_loss:  6.121237929662069
count_amostra: 3612


Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors


iteration:  330 , total_loss:  6.0501412073771155
count_amostra: 3972
iteration:  360 , total_loss:  5.8644900957743324
count_amostra: 4332
iteration:  390 , total_loss:  5.713110526402791
count_amostra: 4692
iteration:  420 , total_loss:  5.563763443628947
count_amostra: 5052
iteration:  450 , total_loss:  5.4156730969746905
count_amostra: 5412
iteration:  480 , total_loss:  5.33568704922994
count_amostra: 5772
iteration:  510 , total_loss:  5.191839154561361
count_amostra: 6132
iteration:  540 , total_loss:  4.979401350021362
count_amostra: 6492
iteration:  570 , total_loss:  4.799485445022583
count_amostra: 6852
iteration:  600 , total_loss:  4.669345998764038
count_amostra: 7212
iteration:  630 , total_loss:  4.574394702911377
count_amostra: 7572
iteration:  660 , total_loss:  4.463070487976074
count_amostra: 7932
iteration:  690 , total_loss:  4.334119288126628
count_amostra: 8292
iteration:  720 , total_loss:  4.3181459426879885
count_amostra: 8652
iteration:  750 , total_loss:  

In [ ]:
53s

In [6]:

#convert to huggingface Transformers

from transformers import BertConfig, BertForMaskedLM, BertTokenizerFast

config = BertConfig(
    vocab_size=VOCAB_SIZE+8,
    max_position_embeddings=MODEL_MAX_SEQ_LEN,
    num_attention_heads=12,
    num_hidden_layers=12,
    type_vocab_size=2,
)

bert_model=BertForMaskedLM(config=config)
bert_model.bert=model

In [7]:
bert_model.save_pretrained('models/bert')

## Evaluate

### QA

### Trainer hf

In [8]:
from datasets import load_dataset

dataset = load_dataset("squad", split="train[:5000]")
dataset = dataset.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1000
    })
})

In [9]:
max_length = MODEL_MAX_SEQ_LEN#384
stride = 128
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [10]:
validation_dataset = dataset["test"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=dataset["test"].column_names,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map: 100%|██████████| 1000/1000 [00:00<00:00, 4974.70 examples/s]


In [11]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=tokenizer.model_max_length,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        # print('sequence_ids: ',sequence_ids)
        # print('sequence_ids: ',len(sequence_ids))

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1 and idx < len(sequence_ids) - 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [12]:
from transformers import DefaultDataCollator

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names,batch_size=4)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 2587.68 examples/s]


In [13]:
from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

#BertForMaskedLM
from transformers import BertForMaskedLM

args = TrainingArguments(
    "bert-finetuned-squad",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
)
# bert_model = BertForMaskedLM.from_pretrained(pretrained_model_name_or_path='bert-base-uncased')
# bert_model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")
# bert_model = AutoModelForQuestionAnswering.from_pretrained("/home/kiki/dados_servidor/Servidor/NLPLab/berts/models/bert_spanbert")
if 'miu' in cur_dir:
    bert_model = AutoModelForQuestionAnswering.from_pretrained("/home/miu/Projects/NLPLab/berts/models/bert")
else:
    bert_model = AutoModelForQuestionAnswering.from_pretrained("/home/kiki/dados_servidor/Servidor/NLPLab/berts/models/bert")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at /home/miu/Projects/NLPLab/berts/models/bert and are newly initialized: ['bert.encoder.layer.3.intermediate.dense.weight', 'bert.encoder.layer.9.attention.output.LayerNorm.bias', 'bert.encoder.layer.6.attention.output.dense.bias', 'bert.encoder.layer.1.attention.output.dense.weight', 'bert.encoder.layer.2.attention.self.value.bias', 'bert.encoder.layer.8.attention.output.LayerNorm.bias', 'bert.encoder.layer.10.attention.output.dense.weight', 'bert.encoder.layer.8.output.LayerNorm.weight', 'bert.encoder.layer.6.attention.output.dense.weight', 'bert.encoder.layer.2.attention.self.query.weight', 'bert.encoder.layer.4.intermediate.dense.weight', 'bert.encoder.layer.4.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.9.attention.self.value.bias', 'bert.encoder.layer.5.attention.self.query.weight', 'qa_outputs.bias', 'bert.encoder.layer.8.attention.sel

In [14]:
trainer = Trainer(
    model=bert_model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: abcp4. Use `wandb login --relogin` to force relogin


 20%|██        | 500/2500 [00:39<02:38, 12.61it/s]

{'loss': 4.7581, 'learning_rate': 1.6000000000000003e-05, 'epoch': 1.0}


 40%|████      | 1000/2500 [01:20<01:57, 12.82it/s]

{'loss': 4.0755, 'learning_rate': 1.2e-05, 'epoch': 2.0}


...

100%|██████████| 2500/2500 [03:21<00:00, 12.80it/s]

{'loss': 3.3212, 'learning_rate': 8e-09, 'epoch': 5.0}


100%|██████████| 2500/2500 [03:22<00:00, 12.37it/s]

{'train_runtime': 206.7869, 'train_samples_per_second': 96.718, 'train_steps_per_second': 12.09, 'train_loss': 3.8819138671875, 'epoch': 5.0}


TrainOutput(global_step=2500, training_loss=3.8819138671875, metrics={'train_runtime': 206.7869, 'train_samples_per_second': 96.718, 'train_steps_per_second': 12.09, 'train_loss': 3.8819138671875, 'epoch': 5.0})

In [15]:
from tqdm.auto import tqdm
import collections
import numpy as np
import evaluate

metric = evaluate.load("squad")
n_best = 20
max_answer_length = 30
predicted_answers = []

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [16]:
import evaluate

predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, dataset["test"])

100%|██████████| 1000/1000 [00:00<00:00, 1607.05it/s]


{'exact_match': 3.9, 'f1': 8.451318538500729}

100%|██████████| 1000/1000 [00:00<00:00, 1573.49it/s]


{'exact_match': 4.2, 'f1': 8.794485888565596}

In [ ]:
#100k 512 tokens
{'exact_match': 4.2, 'f1': 8.794485888565596}

## SEQ

In [6]:
# hidden layer extractor class, for magically adding adapter to language model to be pretrained
class HiddenLayerExtractor(nn.Module):
    def __init__(self, net, layer = -2):
        super().__init__()
        self.net = net
        self.layer = layer

        self.hidden = None
        self.hook_registered = False

    def _find_layer(self):
        if type(self.layer) == str:
            modules = dict([*self.net.named_modules()])
            return modules.get(self.layer, None)
        elif type(self.layer) == int:
            children = [*self.net.children()]
            return children[self.layer]
        return None

    def _hook(self, _, __, output):
        self.hidden = output

    def _register_hook(self):
        layer = self._find_layer()
        assert layer is not None, f'hidden layer ({self.layer}) not found'
        handle = layer.register_forward_hook(self._hook)
        self.hook_registered = True

    def forward(self, x):
        if self.layer == -1:
            return self.net(x)

        if not self.hook_registered:
            self._register_hook()

        _ = self.net(x)
        hidden = self.hidden
        self.hidden = None
        assert hidden is not None, f'hidden layer {self.layer} never emitted an output'
        return hidden

In [7]:
model_extractor = HiddenLayerExtractor(model,-2)

In [8]:
import transformers
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import os
from torch.utils.data import  DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer

class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # self.l1 = model._modules['model']
        self.l1 = model_extractor
        self.l2 = torch.nn.Dropout(0.3)
        self.fl = torch.nn.Flatten()
        self.l3 = torch.nn.Linear(768*(MODEL_MAX_SEQ_LEN-2), 3)
    
    def forward(self, ids):
        output= self.l1(ids)
        # print('l1:',output.shape)
        output = self.l2(output)
        # print('l2:',output.shape)
        output = self.fl(output)
        # print('fl:',output.shape)
        output = self.l3(output)
        # print('l3:',output.shape)
        return output

classifier_model = Classifier()
classifier_model=classifier_model.to('cuda')
optimizer = torch.optim.Adam(params =  classifier_model.parameters(), lr=2e-5,weight_decay=0.01)

In [9]:
task='mrpc'
dataset = load_dataset('glue', task, split='train')

In [10]:
import numpy as np
class TokenizedDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.ls=[]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        s=norm.normalize_str(self.dataset["sentence1"][i])+'[SEP]'+norm.normalize_str(self.dataset["sentence2"][i])
        tokens = vocab.tokenize(s).tolist()
        
        self.ls.append(i)
        
        #trucate
        tokens=tokens[:MODEL_MAX_SEQ_LEN - 2]
        l=len(tokens)
        for j in range(l,MODEL_MAX_SEQ_LEN - 2):
            tokens.append(0)
        tokens=torch.Tensor(tokens).long()

        label=self.dataset["label"][i]
        
        d={'input_ids':tokens,'label':label}
        return d



from torch.utils.data import DataLoader

tokenized_dataset = TokenizedDataset(dataset)
# train_dataloader=CustomDataloader(tokenized_dataset)
train_dataloader = DataLoader(
        tokenized_dataset, batch_size=DEVICE_BATCH_SIZE
    )

In [11]:
import numpy as np
from torch.autograd import Variable

classifier_model.train()

def loss_fn(outputs, targets):
    loss=torch.nn.CrossEntropyLoss()(outputs, targets)
    return loss

In [12]:
#FINETUNING

epochs=5
running_loss=0
for i in range(epochs):
    for _,data in enumerate(train_dataloader):
        ids = data['input_ids'].to('cuda', dtype = torch.long)
        targets = data['label'].to('cuda', dtype = torch.long)

        outputs = classifier_model(ids)
        
        loss = loss_fn(outputs,targets)
        # loss=outputs.loss

        optimizer.zero_grad()
        running_loss+=loss.item()
        if _%50==0:
            print(f'Epoch: {i}, Loss:  {running_loss/50}')
            running_loss=0
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch: 0, Loss:  0.030146934986114503
Epoch: 1, Loss:  0.5000924682617187
Epoch: 2, Loss:  0.5059760808944702
Epoch: 3, Loss:  0.4948312652111053
Epoch: 4, Loss:  0.48995151281356814


In [13]:
#EVALUATE
dataset = load_dataset('glue', task, split='validation')
tokenized_dataset = TokenizedDataset(dataset)

eval_dataloader = DataLoader(
        tokenized_dataset, batch_size=DEVICE_BATCH_SIZE
    )

In [14]:
model.eval()
predictions=[]
labels=[]
c=0
for _,data in enumerate(eval_dataloader):
    ids = data['input_ids'].to('cuda', dtype = torch.long)
    targets = data['label'].to('cuda', dtype = torch.long)
    
    outputs = classifier_model(ids)
    
    outputs=torch.argmax(outputs, dim=1).cpu()
    targets=targets.cpu()
    
    outputs=[int(o.item()) for o in outputs]
    targets=[int(o.item()) for o in targets]

    predictions+=outputs
    labels+=targets

In [15]:
from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.60      0.07      0.12       129
           1       0.69      0.98      0.81       279

    accuracy                           0.69       408
   macro avg       0.65      0.52      0.47       408
weighted avg       0.66      0.69      0.60       408



In [ ]:
#Bert 3 layers  10000 dataset
#               precision    recall  f1-score   support

#            0       0.61      0.09      0.15       129
#            1       0.70      0.97      0.81       279

#     accuracy                           0.69       408
#    macro avg       0.65      0.53      0.48       408
# weighted avg       0.67      0.69      0.60       408